# Crossing Paths

The purpose of this juptyer notebook is to take two files of location history downloaded from google as a .json file to find the closest two people came to eachother. This notebook was created within the context of the coronavirus and identifying how may have had contact with someone that is infected. However, calculating how close two people have come to each other over a given time period could be a pair activity in a math classroom.

### The Input File Specification

The following example uses two different possible file inputs. Both are obtained from Google location data. To obtain this data, the user must download Google maps and allow it to use their location at all times. More detailed instructions can be found here: https://support.google.com/accounts/answer/3024190?hl=en. The notebook is tailored to work with a .json file and extract data to compile data that resembles
```
Time Latitude Longitude Date String Time Stamps

0.000000,	-0.081457,	-0.065700,	-0.114250,	0.154935,
0.009988,	-0.081636,	-0.057592,	-0.085180,	0.131289,
0.019976,	-0.151881,	-0.028972,	-0.128110,	0.200797,
0.029963,	-0.145120,	-0.014326,	-0.186789,	0.236971,
0.039951,	-0.052697,	-0.040153,	-0.211472,	0.221607
```

### A Note
See the PSM GPS Plotly Mapping Example - Mapbox.ipynb for more information on how to set up an account on plotly and mapbox to plot location data later in the notebook, you will need to input your username and key below.

In [3]:
import sys
from pathlib import Path

import json
from zipfile import ZipFile
import datetime as dt
from datetime import datetime
import pandas as pd
import numpy as np

#import chart_studio.plotly as py
import chart_studio.plotly as py
import plotly.graph_objs as go
from haversine import haversine, Unit

#load plotly creds
from chart_studio.tools import set_credentials_file
from haversine import haversine, Unit
from creds import *
set_credentials_file(username='username', api_key='key')

In [4]:
# Open pickle file
person_a = pd.read_pickle("LocationData.pkl")

In [6]:
# Google Location data file name, optionally read a zipped data file.
datapath = Path((r'C:\Users\name\OneDrive\Documents\2019-2020\Project\GPS\Crossing Paths'))
datafile = 'Location History.json'

# Read zip or not
zipped = False

if zipped:
    datalocation = datapath/(datafile+".zip")
    zf = ZipFile(datalocation, 'r')
    zhandle = zf.open(datafile)
    history = json.load(zhandle)
else:
    # Load GPS data from JSON file
    history = json.load(open(datapath/datafile)) 

# Load the location data points
points = history['locations']

In [7]:
# Collect timestamps, lat, lon from JSON, build dataframe
dates = []
lats = []
lons = []
time_stamps = []

# Create empty dataframe
person_b = pd.DataFrame()

for i in range(len(points)):
    dates.append(dt.datetime.fromtimestamp(int(points[i]['timestampMs'])/1000)) 
    lats.append(points[i]['latitudeE7']/1.e7)
    lons.append(points[i]['longitudeE7']/1.e7)
    time_stamps.append(datetime.timestamp(dates[i]))
    
person_b['Time'] = dates
person_b['Latitude'] = lats
person_b['Longitude'] = lons
person_b = person_b.set_index('Time')
person_b['Date String'] = person_b.index.strftime('%m/%d/%Y %H:%M')
person_b['Time Stamps'] = time_stamps


In [8]:
# Retrieving just the data fromd dates of interest
a_date = person_a["2020-04-16":"2020-04-20"]
b_date = person_b["2020-04-16":"2020-04-20"]
a_date.head(20)
b_date.head(20)

,Latitude,Longitude,Date String,Time Stamps
Time,,,,
2020-04-16 00:37:51.995,46.072145,-118.334272,04/16/2020 00:37,1.587023e+09
2020-04-16 01:47:49.002,46.072154,-118.334242,04/16/2020 01:47,1.587027e+09
2020-04-16 03:13:40.001,46.072060,-118.334107,04/16/2020 03:13,1.587032e+09
2020-04-16 04:28:13.999,46.072412,-118.333982,04/16/2020 04:28,1.587036e+09
2020-04-16 05:08:05.999,46.072201,-118.334208,04/16/2020 05:08,1.587039e+09
2020-04-16 05:57:05.414,46.072112,-118.334256,04/16/2020 05:57,1.587042e+09
2020-04-16 06:12:01.000,46.072244,-118.334083,04/16/2020 06:12,1.587043e+09
2020-04-16 06:35:36.050,46.072194,-118.334137,04/16/2020 06:35,1.587044e+09
2020-04-16 07:17:38.000,46.072123,-118.334087,04/16/2020 07:17,1.587047e+09


In [10]:
# Grab the original index datetimes
a_oi = a_date.index
b_oi = b_date.index

# Now we need uniform indices based on a certain time interval
# Generates 3 minute uniform intervals for april 16th through the end of april 19th
uni_index = pd.date_range(dt.date(2020,4,16), dt.date(2020,4,20), freq = '3T')

# Now we interpolate so every uniform time interval has latitude/longitude data and drop original indexes
a_apr16_3min = a_date.reindex(a_oi.union(uni_index)).interpolate('index').reindex(uni_index)
b_apr16_3min = b_date.reindex(b_oi.union(uni_index)).interpolate('index').reindex(uni_index)

### The Haversine Formula 
Here we use the Haversine formula to calculate the distance between two people given their latitude and longitude coordinates. The curvature of the Earth, or another sphere, the distance from one point to another point is impacted by the curvature which is why we use the Haversine Formula.As a reminder, latitude measures distance between North and South poles whereas longitude measures distance around the equator. The Haversine formula is defined as follows: $$d = 2r\arcsin\sqrt{\sin^2\bigg(\frac{\phi_2-\phi_1}{2}\bigg)+\cos(\phi_1)\cdot\cos(\phi_2)\cdot \sin^2\bigg(\frac{\psi_2-\psi_1}{2}\bigg)}$$ where $r$ is the radius of the Earth, $\phi$ is latitude, and $\psi$ is longitude.

In [12]:
a_lat= a_apr16_3min['Latitude']
b_lat= b_apr16_3min['Latitude']

a_lon= a_apr16_3min['Longitude']
b_lon= b_apr16_3min['Longitude']

b_time= b_apr16_3min['Time Stamps']

violation = 0
for i in range(len(a_lat)):
    a_latlon = (a_lat[i],a_lon[i])
    b_latlon = (b_lat[i],b_lon[i])
    if haversine(a_latlon, b_latlon, unit = Unit.FEET) <= 50:
        violation += 1
if violation != 1:
    print("There were", violation, "contacts.")
    
min_dist = 10000
for i in range(len(a_lat)):
    a_latlon = (a_lat[i],a_lon[i])
    b_latlon = (b_lat[i],b_lon[i])
    b_date = b_time[i]
    if haversine(a_latlon, b_latlon, unit = Unit.FEET) <= min_dist:
        min_dist = haversine(a_latlon, b_latlon, unit = Unit.FEET)
        a_min = a_latlon
        b_min = b_latlon
        time_min = b_date

dt_object = datetime.fromtimestamp(time_min)
print("The closest you were was", round(min_dist,2), "feet from each other at", dt_object)
print("Ally was at", a_min)
print("Schueller was as at", b_min)

There were 0 contacts.
The closest you were was 736.23 feet from each other at 2020-04-19 16:18:00
Ally was at (46.0240268, -118.3057285)
Schueller was as at (46.025952740484946, -118.30486026875448)


### Mapping
The following code produces a map of the two people's interpolated location data and highlights where they came closest. The PSM GPS Plotly Mapping Example - Mapbox.ipynb file should provide instructions for using Plotly and Mapbox to produce this map. The user will need to input their own unsername and key in the first cell of code that imports library. They will also need to input their accesstoken below.

In [14]:
# Create map comparing the orig data (blue) to the interpolated data (red)

smb = []

# plot the uniformly interpolated data
s_plot = a_apr16_3min
smb.append(go.Scattermapbox(
        lat=s_plot["Latitude"],
        lon=s_plot["Longitude"],
        text=s_plot.index.map(str),
        mode='markers',
        #showlegend=False,
        marker = dict(size=4,color=b_apr16_3min["Time Stamps"], colorscale ='Viridis', showscale=True)
      ))

# plot the uniformly interpolated data of ally
s_plot = b_apr16_3min
smb.append(go.Scattermapbox(
        lat=s_plot["Latitude"],
        lon=s_plot["Longitude"],
        text=s_plot.index.map(str),
        mode='markers',
        #showlegend=False,
        marker = dict(size=6, color=b_apr16_3min["Time Stamps"],colorscale='Viridis',showscale=True)
      ))

a_min_lat = a_min[0]
a_min_lon = a_min[1]

b_min_lat = b_min[0]
b_min_lon = b_min[1]

smb.append(go.Scattermapbox(
    lat=(a_min_lat, b_min_lat),
    lon=(a_min_lon, b_min_lon),
    mode='markers',
    marker = dict(size=8,color='red')
))


layout = go.Layout(
    width=1024,
    height=768,
    mapbox=dict(
        accesstoken='accesstoken',
        bearing=0,
        center=dict(
            lat=s_plot["Latitude"].mean(),lon=s_plot["Longitude"].mean()
        ),
        pitch=0,
        zoom=13,
        #style = 'satellite-streets'
    )
)





fig = go.Figure(data=smb, layout=layout)
fig.show()